<a href="https://colab.research.google.com/github/martha-rincon-lopez/Diplomado/blob/Tareas/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Principal Component Analysis (PCA)**

In [35]:
import pandas as pd
import numpy as np
from google.colab import drive
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

In [36]:
def plot_confusion_matrix(cm, labels):
    fig_cm = px.imshow(cm, labels=dict(x="Predicted", y="Actual", color="Count"),
                       x=labels, y=labels, color_continuous_scale='Viridis', text_auto = True,
                       title="Confusion Matrix")
    fig_cm.update_layout(coloraxis_showscale=False)
    fig_cm.show()

Nuestra base de datos es el conjunto Iris, que contiene las medidas de 150 flores que pertenecen a 3 especies:
1. Setosa
2. Versicolor
3. Virginica

Queremos determinar de forma automática la especie a la que pertenece una determinada flor, a partir de las medidas de 4 atributos: la longitud y anchura de sus pétalos y sépalos.
Tenemos un problema de dimension 4 donde nuestro objetivo target es la Specie

**Data Loading**

In [37]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
%cd /content/drive/My \Drive/Diplomado

/content/drive/My Drive/Diplomado


In [39]:
df = pd.read_csv('iriss.csv')
df.tail()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


Dividimos los datos

In [40]:
# X contiene los valores de las primeras 4 columnas del dataframe y todas las filas
X = df.iloc[:,0:4].values

# y contiene los valores de la columna 4 (species)
y = df.iloc[:,4].values

Tenemos que las medidas de los sépalos y pétalos están expresadas en centímetros y al aplicar esta técnica se asume que los datos tienen una distribución normal. Por lo tanto aplicaremos una normalización con media igual a 0 y varianza igual a 1.

In [41]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**PCA with 2 components**

In [42]:
pca_2d = PCA(n_components = 2)
X_pca_2d = pca_2d.fit_transform(X_scaled)
pca_df_2d = pd.DataFrame(data=X_pca_2d, columns =['PC1', 'PC2'])
pca_df_2d['Species'] = y
pca_df_2d.head()

,PC1,PC2,Species
0,-2.264703,0.480027,setosa
1,-2.080961,-0.674134,setosa
2,-2.364229,-0.341908,setosa
3,-2.299384,-0.597395,setosa
4,-2.389842,0.646835,setosa


In [43]:
fig_2d = px.scatter(pca_df_2d, x ='PC1', y ='PC2', color ='Species', template = 'plotly_white', title = 'PCA - 2 Components')
fig_2d.show()

In [45]:
explained_variance_2d = pca_2d.explained_variance_ratio_
print("Explained Variance Ratio (2D):", explained_variance_2d)

Explained Variance Ratio (2D): [0.72962445 0.22850762]


In [46]:
components = pd.DataFrame(abs(pca_2d.components_), columns = ['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width'])
components

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,0.521066,0.269347,0.580413,0.564857
1,0.377418,0.923296,0.024492,0.066942


**Classification**

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df['Species'], test_size = 0.2, random_state = 7)

In [48]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
pca = PCA(n_components = 2)
X_train_pca_2d = pca.fit_transform(X_train_scaled)
X_test_pca_2d = pca.transform(X_test_scaled)

**Logistic Regression**

In [50]:
lr_pca_2d = LogisticRegression()
lr_pca_2d.fit(X_train_pca_2d, y_train)
lr_pca_2d_pred = lr_pca_2d.predict(X_test_pca_2d)

In [51]:
lr_pca_2d_accuracy = accuracy_score(y_test, lr_pca_2d_pred)
lr_pca_2d_precision = precision_score(y_test, lr_pca_2d_pred, average="micro")
lr_pca_2d_recall = recall_score(y_test, lr_pca_2d_pred, average="micro")
lr_pca_2d_f1 = f1_score(y_test, lr_pca_2d_pred,  average="micro")
lr_pca_2d_report = classification_report(y_test, lr_pca_2d_pred)
print("Logistic Regression PCA 2D Classification Report:")
print(lr_pca_2d_report)

Logistic Regression PCA 2D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.67      0.83      0.74        12
   virginica       0.75      0.55      0.63        11

    accuracy                           0.77        30
   macro avg       0.81      0.79      0.79        30
weighted avg       0.78      0.77      0.76        30



In [52]:
lr_pca_2d_cm = confusion_matrix(y_test, lr_pca_2d_pred)
plot_confusion_matrix(lr_pca_2d_cm, ['Setosa', 'Versicolor', 'Virginica'])

**KNN**

In [53]:
knn_pca_2d = KNeighborsClassifier()
knn_pca_2d.fit(X_train_pca_2d, y_train)
knn_pca_2d_pred = knn_pca_2d.predict(X_test_pca_2d)

In [54]:
knn_pca_2d_accuracy = accuracy_score(y_test, knn_pca_2d_pred)
knn_pca_2d_precision = precision_score(y_test, knn_pca_2d_pred,  average = "micro")
knn_pca_2d_recall = recall_score(y_test, knn_pca_2d_pred,  average = "micro")
knn_pca_2d_f1 = f1_score(y_test, knn_pca_2d_pred,  average = "micro")
knn_pca_2d_report = classification_report(y_test, knn_pca_2d_pred)
print("KNN PCA 2D Classification Report:")
print(knn_pca_2d_report)

KNN PCA 2D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.73      0.92      0.81        12
   virginica       0.88      0.64      0.74        11

    accuracy                           0.83        30
   macro avg       0.87      0.85      0.85        30
weighted avg       0.85      0.83      0.83        30



In [55]:
knn_pca_2d_cm = confusion_matrix(y_test, knn_pca_2d_pred)
plot_confusion_matrix(knn_pca_2d_cm, ['Setosa', 'Versicolor', 'Virginica'])

**SVM**

In [56]:
svm_pca_2d = SVC()
svm_pca_2d.fit(X_train_pca_2d, y_train)
svm_pca_2d_pred = svm_pca_2d.predict(X_test_pca_2d)

In [57]:
svm_pca_2d_accuracy = accuracy_score(y_test, svm_pca_2d_pred)
svm_pca_2d_precision = precision_score(y_test, svm_pca_2d_pred,  average = "micro")
svm_pca_2d_recall = recall_score(y_test, svm_pca_2d_pred,  average = "micro")
svm_pca_2d_f1 = f1_score(y_test, svm_pca_2d_pred,  average = "micro")
svm_pca_2d_report = classification_report(y_test, svm_pca_2d_pred)
print("SVM PCA 2D Classification Report:")
print(svm_pca_2d_report)

SVM PCA 2D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.71      0.83      0.77        12
   virginica       0.78      0.64      0.70        11

    accuracy                           0.80        30
   macro avg       0.83      0.82      0.82        30
weighted avg       0.80      0.80      0.80        30



In [58]:
svm_pca_2d_cm = confusion_matrix(y_test, svm_pca_2d_pred)
plot_confusion_matrix(svm_pca_2d_cm, ['Setosa', 'Versicolor', 'Virginica'])

**Naive Bayes**

In [59]:
nb_pca_2d = GaussianNB()
nb_pca_2d.fit(X_train_pca_2d, y_train)
nb_pca_2d_pred = nb_pca_2d.predict(X_test_pca_2d)

In [60]:
nb_pca_2d_accuracy = accuracy_score(y_test, nb_pca_2d_pred)
nb_pca_2d_precision = precision_score(y_test, nb_pca_2d_pred, average = "micro")
nb_pca_2d_recall = recall_score(y_test, nb_pca_2d_pred, average = "micro")
nb_pca_2d_f1 = f1_score(y_test, nb_pca_2d_pred, average = "micro")
nb_pca_2d_report = classification_report(y_test, nb_pca_2d_pred)
print("Naive Bayes PCA 2D Classification Report:")
print(nb_pca_2d_report)

Naive Bayes PCA 2D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.67      0.83      0.74        12
   virginica       0.75      0.55      0.63        11

    accuracy                           0.77        30
   macro avg       0.81      0.79      0.79        30
weighted avg       0.78      0.77      0.76        30



In [61]:
nb_pca_2d_cm = confusion_matrix(y_test, nb_pca_2d_pred)
plot_confusion_matrix(nb_pca_2d_cm, ['Setosa', 'Versicolor', 'Virginica'])

# **PCA with 3 components**

In [62]:
pca_3d = PCA(n_components = 3)
X_pca_3d = pca_3d.fit_transform(X_scaled)
pca_df_3d = pd.DataFrame(data = X_pca_3d, columns = ['PC1', 'PC2', 'PC3'])
pca_df_3d['Species'] = y
pca_df_3d.head()

,PC1,PC2,PC3,Species
0,-2.264703,0.480027,-0.127706,setosa
1,-2.080961,-0.674134,-0.234609,setosa
2,-2.364229,-0.341908,0.044201,setosa
3,-2.299384,-0.597395,0.091290,setosa
4,-2.389842,0.646835,0.015738,setosa


In [63]:
fig_3d = px.scatter_3d(pca_df_3d, x='PC1', y='PC2', z='PC3', color='Species', template = 'plotly_white', title = 'PCA - 3 Components')
fig_3d.show()

In [64]:
explained_variance_3d = pca_3d.explained_variance_ratio_
print("Explained Variance Ratio (3D):", explained_variance_3d)

Explained Variance Ratio (3D): [0.72962445 0.22850762 0.03668922]


In [65]:
components = pd.DataFrame(abs(pca_3d.components_), columns = ['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width'])
components

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,0.521066,0.269347,0.580413,0.564857
1,0.377418,0.923296,0.024492,0.066942
2,0.719566,0.244382,0.142126,0.634273


**Classification**

In [66]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df['Species'], test_size = 0.2, random_state = 7)

In [67]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [68]:
pca = PCA(n_components = 3)
X_train_pca_3d = pca.fit_transform(X_train_scaled)
X_test_pca_3d = pca.transform(X_test_scaled)

**Logistic Regression**

In [69]:
lr_pca_3d = LogisticRegression()
lr_pca_3d.fit(X_train_pca_3d, y_train)
lr_pca_3d_pred = lr_pca_3d.predict(X_test_pca_3d)

In [70]:
lr_pca_3d_accuracy = accuracy_score(y_test, lr_pca_3d_pred)
lr_pca_3d_precision = precision_score(y_test, lr_pca_3d_pred, average = "micro")
lr_pca_3d_recall = recall_score(y_test, lr_pca_3d_pred, average = "micro")
lr_pca_3d_f1 = f1_score(y_test, lr_pca_3d_pred,  average = "micro")
lr_pca_3d_report = classification_report(y_test, lr_pca_3d_pred)
print("Logistic Regression PCA 3D Classification Report:")
print(lr_pca_3d_report)

Logistic Regression PCA 3D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.83      0.83      0.83        12
   virginica       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.88      0.88      0.88        30
weighted avg       0.87      0.87      0.87        30



In [71]:
lr_pca_3d_cm = confusion_matrix(y_test, lr_pca_3d_pred)
plot_confusion_matrix(lr_pca_3d_cm, ['Setosa', 'Versicolor', 'Virginica'])

**KNN**

In [72]:
knn_pca_3d = KNeighborsClassifier()
knn_pca_3d.fit(X_train_pca_3d, y_train)
knn_pca_3d_pred = knn_pca_3d.predict(X_test_pca_3d)

In [73]:
knn_pca_3d_accuracy = accuracy_score(y_test, knn_pca_3d_pred)
knn_pca_3d_precision = precision_score(y_test, knn_pca_3d_pred,  average = "micro")
knn_pca_3d_recall = recall_score(y_test, knn_pca_3d_pred,  average = "micro")
knn_pca_3d_f1 = f1_score(y_test, knn_pca_3d_pred,  average = "micro")
knn_pca_3d_report = classification_report(y_test, knn_pca_3d_pred)
print("KNN PCA 3D Classification Report:")
print(knn_pca_3d_report)

KNN PCA 3D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.79      0.92      0.85        12
   virginica       0.89      0.73      0.80        11

    accuracy                           0.87        30
   macro avg       0.89      0.88      0.88        30
weighted avg       0.87      0.87      0.87        30



In [74]:
knn_pca_3d_cm = confusion_matrix(y_test, knn_pca_3d_pred)
plot_confusion_matrix(knn_pca_3d_cm, ['Setosa', 'Versicolor', 'Virginica'])

**SVM**

In [75]:
svm_pca_3d = SVC()
svm_pca_3d.fit(X_train_pca_3d, y_train)
svm_pca_3d_pred = svm_pca_3d.predict(X_test_pca_3d)

In [76]:
svm_pca_3d_accuracy = accuracy_score(y_test, svm_pca_3d_pred)
svm_pca_3d_precision = precision_score(y_test, svm_pca_3d_pred,  average = "micro")
svm_pca_3d_recall = recall_score(y_test, svm_pca_3d_pred,  average = "micro")
svm_pca_3d_f1 = f1_score(y_test, svm_pca_3d_pred,  average = "micro")
svm_pca_3d_report = classification_report(y_test, svm_pca_3d_pred)
print("SVM PCA 3D Classification Report:")
print(svm_pca_3d_report)

SVM PCA 3D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.83      0.83      0.83        12
   virginica       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.88      0.88      0.88        30
weighted avg       0.87      0.87      0.87        30



In [77]:
svm_pca_3d_cm = confusion_matrix(y_test, svm_pca_3d_pred)
plot_confusion_matrix(svm_pca_3d_cm, ['Setosa', 'Versicolor', 'Virginica'])

**Naive Bayes**

In [78]:
nb_pca_3d = GaussianNB()
nb_pca_3d.fit(X_train_pca_3d, y_train)
nb_pca_3d_pred = nb_pca_3d.predict(X_test_pca_3d)

In [79]:
nb_pca_3d_accuracy = accuracy_score(y_test, nb_pca_3d_pred)
nb_pca_3d_precision = precision_score(y_test, nb_pca_3d_pred, average = "micro")
nb_pca_3d_recall = recall_score(y_test, nb_pca_3d_pred, average = "micro")
nb_pca_3d_f1 = f1_score(y_test, nb_pca_3d_pred, average = "micro")
nb_pca_3d_report = classification_report(y_test, nb_pca_3d_pred)
print("Naive Bayes PCA 3D Classification Report:")
print(nb_pca_3d_report)

Naive Bayes PCA 3D Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.82      0.75      0.78        12
   virginica       0.75      0.82      0.78        11

    accuracy                           0.83        30
   macro avg       0.86      0.86      0.86        30
weighted avg       0.84      0.83      0.83        30



In [80]:
nb_pca_3d_cm = confusion_matrix(y_test, nb_pca_3d_pred)
plot_confusion_matrix(nb_pca_3d_cm, ['Setosa', 'Versicolor', 'Virginica'])